In [117]:
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [118]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
spacex_df.drop(columns='Unnamed: 0', inplace=True)
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [119]:
spacex_df.head()

,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [120]:
# Create a dash application
app = dash.Dash(__name__)

In [121]:
site_names = list(spacex_df['Launch Site'].unique())

In [122]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                             options = [{'label':'All Sites', 'value':'ALL'},
                                                        {'label':site_names[0], 'value':site_names[0]},
                                                        {'label':site_names[1], 'value':site_names[1]},
                                                        {'label':site_names[2], 'value':site_names[2]},
                                                        {'label':site_names[3], 'value':site_names[3]}],
                                            value='ALL',
                                            placeholder='place holder here',
                                            searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step = 1000,
                                                value=[min_payload, max_payload],
                                                marks={0:'0', 5000:'5000', 10000:'10000'}),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
             Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(data_frame = spacex_df, values='class', names='Launch Site', title = 'Total Success Launches By Site')
        return fig
    else:
        site_df = spacex_df.loc[spacex_df['Launch Site'] == entered_site]
        fig = px.pie(data_frame = site_df, names='class', title = f'Total Success Launches for site {entered_site}')
        return fig
    
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
              Input(component_id='payload-slider',component_property='value')]
              )

def get_scatter_plot(site_name, payload_mass):
    if site_name == 'ALL':
        filtered_df = spacex_df[(spacex_df['Payload Mass (kg)']>=payload_mass[0])
                               & (spacex_df['Payload Mass (kg)']<=payload_mass[1])]
        scatter_chart = px.scatter(data_frame = filtered_df, y='class', x='Payload Mass (kg)',
        color='Booster Version Category', title = 'Correlation between Payload and Success for all Sites')                  
        return scatter_chart
    else:
        filtered_df = spacex_df[(spacex_df['Payload Mass (kg)']>=payload_mass[0])
                               & (spacex_df['Payload Mass (kg)']<=payload_mass[1])]
        site_df = filtered_df.loc[filtered_df['Launch Site'] == site_name]
        scatter_chart = px.scatter(data_frame = site_df, y='class', x='Payload Mass (kg)',
        color='Booster Version Category', title = f'Correlation between Payload and Success for site {site_name}')       
        return scatter_chart

if __name__ == '__main__':
    app.run_server(port=8011)